In [5]:
import os
from langchain.chat_models import AzureChatOpenAI # Azure OpenAI を利用する場合
# from langchain.chat_models import ChatOpenAI # OpenAI APIを利用する場合
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
# from langchain.memory import ConversationSummaryBufferMemory
from langchain.utilities import SerpAPIWrapper
# from langchain.memory import CosmosDBChatMessageHistory # for cosmosDB
from langchain.memory.chat_message_histories import RedisChatMessageHistory # for Redis
from langchain.agents import (
    Tool,
    initialize_agent,
    AgentType
)
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder, 
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from dotenv import load_dotenv 
load_dotenv("/home/shohei/.env", override=True)

# AzureChatOpenAIクラスのインスタンス作成
chat = AzureChatOpenAI(openai_api_base = os.environ["AZURE_OPENAI_API_BASE"], # リソースによって異なる
                        openai_api_version = "2023-07-01-preview", 
                        openai_api_key = os.environ["AZURE_OPENAI_API_KEY"],
                        deployment_name="gpt-35-turbo", # モデルのデプロイ名
                        temperature=0.7)

search = SerpAPIWrapper(serpapi_api_key=os.environ["SERPAPI_API_KEY"])
tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]

# 会話の履歴を保持するためのメモリオブジェクトを作成
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)



In [6]:
agent_chain = initialize_agent(tools, chat, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory, )

In [7]:
agent_chain.run(input="今日は仕事つかれたよー")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "お疲れ様です！ゆっくり休んでくださいね。"
}

> Finished chain.


'お疲れ様です！ゆっくり休んでくださいね。'

In [8]:
agent_chain.run(input="Microsoftって何の会社？")



> Entering new AgentExecutor chain...
{
    "action": "Current Search",
    "action_input": "Microsoft"
}
Observation: Explore Microsoft products and services for your home or business. Shop Surface, Microsoft 365, Xbox, Windows, Azure, and more. Find downloads and get ...
Thought:{
    "action": "Final Answer",
    "action_input": "Microsoft is a multinational technology company that develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services."
}

> Finished chain.


'Microsoft is a multinational technology company that develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services.'